In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import DataFrame
from pyspark.sql import Window

from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import make_moons, make_classification, make_regression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from connector import spark_connector

In [ ]:
con = spark_connector(exec_inst=100, exec_cores=5, memory_exec=25, marshal_mode=True)

In [ ]:
con.spark

In [ ]:
X, y = make_classification(
    n_samples=2000000,
    n_features=15,
    n_informative=15,
    n_redundant=0,
    n_repeated=0,
    n_classes=7,
    n_clusters_per_class=2,
    weights=None,
    flip_y=0.04,
    class_sep=1.0,
    hypercube=True,
    shift=0.0,
    scale=1.0,
    shuffle=True,
    random_state=0,
)

In [ ]:
X, y =  make_regression(
    n_samples=2000000,
    n_features=3,
    n_informative=3,
    n_targets=1,
    bias=0.0,
    effective_rank=None,
    noise=0.01,
    shuffle=True,
    coef=False,
    random_state=0,
)

In [ ]:
data = pd.DataFrame(X, columns=[f'x{i+1}' for i in range(X.shape[1])])
data['y'] = y

In [ ]:
train, test = train_test_split(data, random_state=6, test_size=0.1)
train.shape, test.shape

In [ ]:
from sklearn.metrics import classification_report, mean_absolute_percentage_error

In [ ]:
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

In [ ]:
#nn = KNeighborsRegressor(n_neighbors=15, weights='distance', algorithm='brute', n_jobs=-1).fit(train.drop('y', axis=1), train['y'])
nn = KNeighborsClassifier(n_neighbors=15, weights='distance', algorithm='brute', n_jobs=-1).fit(train.drop('y', axis=1), train['y'])

In [ ]:
#mean_absolute_percentage_error(test[:].reset_index(names=['index_test']).sort_values(by='index_test')['y'].to_numpy(), nn.predict(test[:].reset_index(names=['index_test']).sort_values(by='index_test').drop(['y','index_test'], axis=1)))
print(classification_report(test[:].reset_index(names=['index_test']).sort_values(by='index_test')['y'].to_numpy(), nn.predict(test[:].reset_index(names=['index_test']).sort_values(by='index_test').drop(['y','index_test'], axis=1))))

In [ ]:
del nn

In [ ]:
train = con.spark.createDataFrame(train)
test = con.spark.createDataFrame(test.drop(['y'], axis=1))

In [ ]:
from KNN import spark_knn

In [ ]:
spark_knn?

In [ ]:
knn = spark_knn(spark = con.spark, train_df = train_df)

In [ ]:
knn.fit?

In [ ]:
%%time
train2 = knn.fit(window_size=None)

In [ ]:
knn.train_df

In [ ]:
knn.predict?

In [ ]:
result = knn.predict(test_df, pred_df_window_size=None, n_neighbors=15) #40000
pred_y = result.toPandas()

In [ ]:
mean_absolute_percentage_error(test.reset_index(names=['index_test']).sort_values(by='index_test')['y'].to_numpy(), pred_y.sort_values(by='index_test')['y'].to_numpy())

In [ ]:
print(classification_report(test.reset_index(names=['index_test']).sort_values(by='index_test')['y'].to_numpy(), pred_y.sort_values(by='index_test')['y'].to_numpy()))

In [ ]:
con.stop_spark()